In [ ]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as  pd
df=pd.read_json('/content/drive/My Drive/7500docpercat_corpus(1).json',lines=True)

In [ ]:
!pip install clean-text[sklearn] #https://www.analyticsvidhya.com/blog/2021/06/text-preprocessing-in-nlp-with-python-codes/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from cleantext.sklearn import CleanTransformer

cleaner = CleanTransformer(no_punct=False, lower=False)

x=cleaner.transform(['Happily clean your text!', 'Another Input'])

In [ ]:
print (x)

['Happily clean your text!', 'Another Input']


In [ ]:
df['category'] = [''.join(map(str, l)) for l in df['category']]
df['category'].value_counts()

agriculture           7500
coal                  7500
construction          7500
fertilizer            7500
iron ore and steel    7500
maritime              7500
petroleum             7500
Name: category, dtype: int64

In [ ]:
df=pd.read_json('/content/drive/My Drive/7500docpercat_corpus(1).json',lines=True)
df['article'] = [''.join(map(str, l)) for l in df['article']]
print(df['article'][0])

Cargill consolidates food ingredients distribution network in U.S and Canada to six channel partners. MINNEAPOLIS (November 14, 2018) — Cargill's food ingredients and applications business has chosen six North American distribution partners – Univar Inc., Gillco Ingredients, International Food Products Corporation, Batory Foods, Pearson Sales Company and St. Charles Trading, Inc. This move optimizes customer experience, streamlines the supply chain and creates a platform to generate sustainable growth through a select group of committed channel partners. "In today's rapidly changing marketplace, our customers need industry-leading distributor partners who can augment our in-house expertise with broad product portfolios for one-stop convenience, regulatory support, flexible lead times and minimum order quantities," said Pat Rogers, commercial leader in Cargill's North America food ingredients business. "We're confident these partners can deliver on those expectations, bringing added val

In [ ]:
import collections
df=pd.read_json('/content/drive/My Drive/7500docpercat_corpus(1).json',lines=True)
df['text']=df['article']
df["text"] = df["text"].str.lower()
#Remove punctuation
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
df["text"] = df["text"].apply(lambda text: remove_punctuation(text))
#REmove stopwords
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text"] = df["text"].apply(lambda text: remove_stopwords(text))
#REmove frequent words 
from collections import Counter
cnt = Counter()
for text in df["text"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["text"] = df["text"].apply(lambda text: remove_freqwords(text))
#Remove Rare Words
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["text"] = df["text"].apply(lambda text: remove_rarewords(text))
#Lemmatization
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df["text_lemmatized"] = df["text"].apply(lambda text: lemmatize_words(text))
df.head()

,category,article,analyzedTags,extractionsource,tags,text,text_lemmatized
0,[agriculture],Cargill consolidates food ingredients distribu...,[],cargill,[],cargill consolidates food ingredients distribu...,cargill consolidate food ingredient distributi...
1,[agriculture],Cargill Names Florian Schattenmann Vice Presid...,[],cargill,[],cargill names florian schattenmann vice presid...,cargill name florian schattenmann vice preside...
2,[agriculture],Honeysuckle White® expands Thanksgiving tracea...,[],cargill,[],honeysuckle white® expands thanksgiving tracea...,honeysuckle white® expand thanksgiving traceab...
3,[agriculture],ADM and Cargill to help farmers achieve greate...,[],cargill,[],adm cargill help farmers achieve greater profi...,adm cargill help farmer achieve great profitab...
4,[agriculture],Agribusinesses Seek to Modernize Global Agricu...,[],cargill,[],agribusinesses seek modernize global agricultu...,agribusiness seek modernize global agricultura...


In [50]:
df.to_csv('/content/drive/My Drive/cleaned_text_7500.csv')